这个文件是单个文件的准确度，延迟和抖动的计算
不考虑延迟：
Accuracy 的判断标准是：
Correct_i = 1, if (label_i==0 and touch_type_idx_i==0)
            1, if (label_i==1 and touch_type_idx_i==GT)
            0, otherwise
​其中：label_map_TCNN = { 0:"NC",1:"ST", 2:"P", 3:"G"}
GT 是 ground truth（真实的手势类别）
"ST" → touch_type_idx == 1
"P" → touch_type_idx == 2
"G" → touch_type_idx == 3
然后计算准确率：
Accuracy=∑𝑖Correct_𝑖/𝑁

考虑延迟：
计算延迟时间T的方法，用第一个label==1 对应的时间 减去 第一个touch_type_idx ！= 0的时间 得到延迟时间T

在考虑延迟的情况下计算准确率的标准是：
Correct_i = 1, if (label_i==0 and touch_type_idx_(i+T)==0)
            1, if (label_i==1 and touch_type_idx_(i+t)==GT)
            0, otherwise
当i+T out of range 时就停止。
GT 是 ground truth（真实的手势类别）
"ST" → touch_type_idx == 1
"P" → touch_type_idx == 2
"G" → touch_type_idx == 3
然后计算准确率：
Accuracy=∑𝑖Correct_𝑖/𝑁


In [16]:
# 不考虑延迟
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
from IPython.core.display import display, HTML

# 读取 CSV 文件
df = pd.read_csv("../DATA/Labeled_data/0213_2LCNN_Post1_ST/labeled_modelResult_data.csv")
folder_name = "0213_2LCNN_Post1_ST"  # 文件夹名称
# 提取 Ground Truth (GT) 手势类别
gesture_mapping = {"NC": 0, "ST": 1, "P": 2, "G": 3}

gt_label = folder_name.split("_")[-1]  # 提取 "ST", "P", or "G"
gt_idx = gesture_mapping.get(gt_label, -1)  # 获取对应 touch_type_idx

# 计算正确预测的情况
df["correct"] = ((df["label"] == 0) & (df["touch_type_idx"] == 0)) | \
                ((df["label"] == 1) & (df["touch_type_idx"] == gt_idx))

accuracy = df["correct"].mean()
y_true = df["label"].replace({0: 0, 1: gt_idx})  # 0 表示无触碰, 1 表示当前手势类别
y_pred = df["touch_type_idx"]

precision = precision_score(y_true, y_pred, average="macro")
recall = recall_score(y_true, y_pred, average="macro")
f1 = f1_score(y_true, y_pred, average="macro")



# 输出结果

print(f"Accuracy: {accuracy:.2%}")
print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")
print(f"F1-score: {f1:.2%}")
# df
# df[(df['label'] == 1) & (df['touch_type_idx'] != 1)]
# displaydf = df[['time', 'label', 'touch_type_idx', 'correct']]
# display(HTML(displaydf.to_html(max_rows=500))) 


/var/folders/md/q4f_fj2d0xg_bb3d94c5z6vc0000gn/T/ipykernel_10355/3608394370.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Accuracy: 90.32%
Precision: 33.51%
Recall: 33.64%
F1-score: 33.29%


/Users/shangyu/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
# 考虑延迟的情况下计算准确率
import pandas as pd

# 读取数据
df = pd.read_csv("../DATA/Labeled_data/0213_2LCNN_Post1_ST/labeled_modelResult_data.csv")

length = len(df)

# 提取 Ground Truth (GT) 手势类别
gesture_mapping = {"NC": 0, "ST": 1, "P": 2, "G": 3}
folder_name = "0213_2LCNN_Post1_ST"  # 文件夹名称
gt_label = folder_name.split("_")[-1]  # 提取 "ST", "P", or "G"
gt_idx = gesture_mapping.get(gt_label, -1)  # 获取对应 touch_type_idx

# 计算延迟时间 T
Time_label, Time_model = None, None
index_label_time, index_model_time = None, None

# 找到第一个 label == 1 的时间
for i in range(length):
    if df.loc[i, "label"] == 1:
        Time_label = df.loc[i, "time"]
        index_label_time = i
        print(f"index_label_time is {i}")
        break  # 只取第一个

# 找到第一个 touch_type_idx != 0 的时间
for i in range(length):
    if df.loc[i, "touch_type_idx"] != 0:
        Time_model = df.loc[i, "model_result_time"]
        index_model_time = i
        print(f"index_model_time is {i}")
        break  # 只取第一个

# 确保找到了两个时间点
if Time_label is None or Time_model is None:
    raise ValueError("未找到 label == 1 或 touch_type_idx != 0 的时间点")

# 计算延迟时间 T
T_delay = Time_model - Time_label
index_diff = index_model_time - index_label_time  # 计算索引偏移量

# 计算正确预测的情况
correct_predictions = []
for i in range(length - index_diff):
    if i + index_diff >= length:
        break  # 超出索引范围时停止

    label_i = df.loc[i, "label"]
    touch_type_idx_iT = df.loc[i + index_diff, "touch_type_idx"]

    if (label_i == 0 and touch_type_idx_iT == 0) or (label_i == 1 and touch_type_idx_iT == gt_idx):
        correct_predictions.append(1)
    else:
        correct_predictions.append(0)

# 计算准确率
accuracy = sum(correct_predictions) / len(correct_predictions) if correct_predictions else 0
print(f"Accuracy (considering delay): {accuracy:.2%}")
print(f"delayed time is {T_delay}")
print(f"the delayed index amount is {index_diff}")

# 计算抖动
import numpy as np
jitter_count = np.sum(np.diff(df['touch_type_idx'].to_numpy()) != 0)# 统计变化次数

print(f"抖动次数: {jitter_count}")


index_label_time is 1013
index_model_time is 1559
Accuracy (considering delay): 89.41%
delayed time is 2.729235987062566
the delayed index amount is 546


In [15]:
# 计算抖动
import numpy as np
# array = np.array((0,0,1,1,1,1,2,2,2,3,3,0,0,1,2,3,0))
jitter_count = np.sum(np.diff(df['touch_type_idx'].to_numpy()) != 0)# 统计变化次数
# jitter_count = np.sum(np.diff(array) != 0)# 统计变化次数

print(f"抖动次数: {jitter_count}")

抖动次数: 22
